#### Tried this pre-process technique, possible to use. But my laptop will explode due to it being not capable of handling and processing this data. This code isn't possible for my laptop.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import os
import gc

In [2]:
DATA_PATH = '../data/'
CHUNK_SIZE = 50000  # Process data in chunks to manage memory

##### Below cell combines all the files into a combined file and also Excluded_labels into single label named anomoly.
This takes up a lot of space in C drive due to pandas temp storage. DON'T RUN IT AGAIN

In [4]:
OUTPUT_PATH = DATA_PATH+'/combined/combined_raw.csv'
# Excluded / rare labels → will be merged as "Anomoly"
EXCLUDE_LABELS = [
    'DoS attacks-Slowloris',
    'DoS attacks-GoldenEye',
    'DDOS attack-LOIC-UDP',
    'Brute Force -Web',
    'Brute Force -XSS',
    'SQL Injection'
]

def get_file_paths(data_path):
    """Get all CSV file paths except the corrupted one"""
    file_paths = []
    for root, _, filenames in os.walk(data_path):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_paths.append(os.path.join(root, filename))
    # remove the corrupted file
    bad_file = os.path.join(DATA_PATH, '02-20-2018.csv')
    if bad_file in file_paths:
        file_paths.remove(bad_file)
    return sorted(file_paths)

# Get file paths
file_paths = get_file_paths(DATA_PATH)
print(f"Found {len(file_paths)} CSV files")

# Read + clean + combine
df_list = []
for file_path in file_paths:
    print(f"Loading: {os.path.basename(file_path)}")
    chunk = pd.read_csv(file_path, low_memory=False)
    
    # Drop timestamp if exists
    chunk = chunk.drop(columns=['Timestamp'], errors='ignore')
    
    # Remove header rows accidentally read as data
    chunk = chunk[chunk['Label'] != 'Label']
    
    # Map excluded labels to "Anomoly"
    chunk['Label'] = chunk['Label'].apply(
        lambda x: 'Anomoly' if x in EXCLUDE_LABELS else x
    )
    
    df_list.append(chunk)

# Combine all dataframes
combined_df = pd.concat(df_list, ignore_index=True)
print(f"Combined dataset shape: {combined_df.shape}")

# Save to CSV
combined_df.to_csv(OUTPUT_PATH, index=False)
print(f"Combined raw-cleaned dataset saved to: {OUTPUT_PATH}")

Found 9 CSV files
Loading: 02-14-2018.csv
Loading: 02-15-2018.csv
Loading: 02-16-2018.csv
Loading: 02-21-2018.csv
Loading: 02-22-2018.csv
Loading: 02-23-2018.csv
Loading: 02-28-2018.csv
Loading: 03-01-2018.csv
Loading: 03-02-2018.csv
Combined dataset shape: (8284195, 79)
Combined raw-cleaned dataset saved to: ../data//combined/combined_raw.csv


In [3]:
dataset = pd.read_csv('..\data\combined\combined_raw.csv')
dataset.head(3)

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,112641719,3,0,0,0.0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,56320859.5,139.300036,56320958.0,56320761.0,Benign
1,0,0,112641466,3,0,0,0.0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,56320733.0,114.551299,56320814.0,56320652.0,Benign
2,0,0,112638623,3,0,0,0.0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,56319311.5,301.934596,56319525.0,56319098.0,Benign


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8284195 entries, 0 to 8284194
Data columns (total 79 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           int64  
 1   Protocol           int64  
 2   Flow Duration      int64  
 3   Tot Fwd Pkts       int64  
 4   Tot Bwd Pkts       int64  
 5   TotLen Fwd Pkts    int64  
 6   TotLen Bwd Pkts    float64
 7   Fwd Pkt Len Max    int64  
 8   Fwd Pkt Len Min    int64  
 9   Fwd Pkt Len Mean   float64
 10  Fwd Pkt Len Std    float64
 11  Bwd Pkt Len Max    int64  
 12  Bwd Pkt Len Min    int64  
 13  Bwd Pkt Len Mean   float64
 14  Bwd Pkt Len Std    float64
 15  Flow Byts/s        float64
 16  Flow Pkts/s        float64
 17  Flow IAT Mean      float64
 18  Flow IAT Std       float64
 19  Flow IAT Max       float64
 20  Flow IAT Min       float64
 21  Fwd IAT Tot        float64
 22  Fwd IAT Mean       float64
 23  Fwd IAT Std        float64
 24  Fwd IAT Max        float64
 25  Fwd IAT Min       

#### Combining and scaling all the values into single csv

In [8]:
import os
print(os.getcwd())

d:\VII\ANT\preprocess


In [9]:
# Optimized function to get all CSV files
def get_file_paths(data_path):
    """Get all CSV file paths efficiently"""
    file_paths = []
    
    for root, _, filenames in os.walk(data_path):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_paths.append(os.path.join(root, filename))
    file_paths.remove(DATA_PATH+'02-20-2018.csv')
    print(f"Found {len(file_paths)} CSV files")
    return sorted(file_paths)

# Optimized data loading function
def load_and_clean_data(file_paths):
    """Load and clean data from multiple CSV files efficiently"""
    # Labels to exclude (too few samples, sum as 'Anomoly')
    exclude_labels = ['DoS attacks-Slowloris', 'DoS attacks-GoldenEye', 'DDOS attack-LOIC-UDP', 
                     'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection']
    
    # Initialize label encoder
    label_encoder = LabelEncoder()
    all_labels = []
    
    # First pass: collect all unique labels
    print("First pass: Collecting unique labels...")
    for file_path in file_paths:
        print(f"Scanning: {os.path.basename(file_path)}")
        # Read only the Label column to save memory
        labels = pd.read_csv(file_path, usecols=['Label'], low_memory=False)
        all_labels.extend(labels['Label'].unique())
    
    # Get unique labels and fit encoder
    unique_labels = list(set(all_labels) - set(exclude_labels))
    
    if 'Label' in unique_labels:
        unique_labels.remove('Label')
        
    excluded_found = any(label in all_labels for label in exclude_labels)
    if excluded_found:
        unique_labels.append('Anomoly')
        
    print(unique_labels)
    label_encoder.fit(unique_labels)
    print(f"Found {len(unique_labels)} unique attack types")
    
    # Second pass: load and process data
    processed_chunks = []
    
    for file_path in file_paths:
        print(f"Processing: {os.path.basename(file_path)}")
        
        # Read in chunks to manage memory
        chunk_reader = pd.read_csv(file_path, chunksize=CHUNK_SIZE, low_memory=False)
        
        for chunk in chunk_reader:
            # Remove timestamp and excluded labels
            chunk = chunk.drop(columns=['Timestamp'], errors='ignore')
            
            if len(chunk) > 0:
                
                chunk = chunk[chunk['Label'] != 'Label']
                chunk['Label'] = chunk['Label'].apply(lambda x: 'Anomoly' if x in exclude_labels else x)
                
                # Encode labels
                chunk['Label'] = label_encoder.transform(chunk['Label'])
                
                # Convert to numeric and clean
                chunk = chunk.apply(pd.to_numeric, errors='coerce')
                chunk = chunk.replace([np.inf, -np.inf], np.nan)
                chunk = chunk.dropna()
                
                processed_chunks.append(chunk)

                if len(processed_chunks) % 10 == 0:  # Every 10 chunks
                    gc.collect()
        
        # Clear memory
        gc.collect()
    
    # Combine all chunks
    print("Combining all processed chunks...")
    combined_df = pd.concat(processed_chunks, ignore_index=True)
    del processed_chunks
    gc.collect()
    
    return combined_df, label_encoder

# Get file paths
file_paths = get_file_paths(DATA_PATH)

# Load and clean data
print("Loading and cleaning data...")
df, label_encoder_fitted = load_and_clean_data(file_paths)

_=gc.collect()

Found 9 CSV files
Loading and cleaning data...
First pass: Collecting unique labels...
Scanning: 02-14-2018.csv
Scanning: 02-15-2018.csv
Scanning: 02-16-2018.csv
Scanning: 02-21-2018.csv
Scanning: 02-22-2018.csv
Scanning: 02-23-2018.csv
Scanning: 02-28-2018.csv
Scanning: 03-01-2018.csv
Scanning: 03-02-2018.csv
['FTP-BruteForce', 'DoS attacks-SlowHTTPTest', 'Infilteration', 'DDOS attack-HOIC', 'Benign', 'DoS attacks-Hulk', 'Bot', 'SSH-Bruteforce', 'Anomoly']
Found 9 unique attack types
Processing: 02-14-2018.csv
Processing: 02-15-2018.csv
Processing: 02-16-2018.csv
Processing: 02-21-2018.csv
Processing: 02-22-2018.csv
Processing: 02-23-2018.csv
Processing: 02-28-2018.csv
Processing: 03-01-2018.csv
Processing: 03-02-2018.csv
Combining all processed chunks...


In [10]:
print("Label encoder mapping:")
for i, label in enumerate(label_encoder_fitted.classes_):
    print(f"{i}: {label}")

Label encoder mapping:
0: Anomoly
1: Benign
2: Bot
3: DDOS attack-HOIC
4: DoS attacks-Hulk
5: DoS attacks-SlowHTTPTest
6: FTP-BruteForce
7: Infilteration
8: SSH-Bruteforce


In [ ]:
print("Saving as CSV...")
mapping_df = pd.DataFrame({
    'encoded_value': range(len(label_encoder_fitted.classes_)),
    'label_name': label_encoder_fitted.classes_
})
mapping_df.to_csv(DATA_PATH+'label_mapping.csv', index=False)
print("Mapping saved as "+DATA_PATH+'label_mapping.csv')

Saving as CSV...
Mapping saved as ../data/data.csv


In [12]:
# Display data information
print(f"Total dataset shape: {df.shape}")
print(f"Number of classes: {df['Label'].nunique()}")
print("Label distribution:")
print(df['Label'].value_counts().sort_index())
print(f"Memory usage: {df.memory_usage().sum() / 1024**2:.2f} MB")

Total dataset shape: (8247888, 79)
Number of classes: 9
Label distribution:
Label
0      55156
1    6077145
2     286191
3     686012
4     461912
5     139890
6     193354
7     160639
8     187589
Name: count, dtype: int64
Memory usage: 4971.19 MB


In [ ]:
class DataPreprocessor:
    def __init__(self, label_encoder=None):
        self.scaler = StandardScaler()
        self.label_encoder = label_encoder
        self.feature_columns = None
        self.is_fitted = False
        
    def fit(self, df):
        """Fit preprocessor on training data"""
        # Identify feature columns (exclude Label)
        self.feature_columns = [col for col in df.columns if col != 'Label']
        
        # Fit scaler on numeric features
        self.scaler.fit(df[self.feature_columns])
        self.is_fitted = True
        
    def transform(self, df):
        """Transform data"""
        if not self.is_fitted:
            raise ValueError("Preprocessor must be fitted before transform")
            
        # Select features
        X = df[self.feature_columns]
        
        # Scale features
        X_scaled = self.scaler.transform(X)
        df_scaled = df.copy()
        df_scaled[self.feature_columns] = X_scaled
        
        return df_scaled
    
# Initialize preprocessor with fitted label encoder
preprocessor = DataPreprocessor(label_encoder=label_encoder_fitted)

# Fit on data
preprocessor.fit(df)

# Transform data
df_scaled = preprocessor.transform(df)
X = df_scaled.drop('Label', axis=1)
y = df_scaled['Label']

print(f"Scaled features shape: {X.shape}")
print(f"Number of classes: {len(np.unique(y))}")
print(f"Classes: {np.unique(y)}")

# Memory cleanup
del X, y
gc.collect()

In [ ]:
# Save cleaned data for faster future loading
n_parts = 1
part_size = len(df_scaled) // n_parts

for i in range(n_parts):
    start_idx = i * part_size
    if i == n_parts - 1:  # Last part gets any remaining rows
        end_idx = len(df_scaled)
    else:
        end_idx = (i + 1) * part_size
    
    part = df_scaled.iloc[start_idx:end_idx]
    save_path = f'/kaggle/working/data_part_{i+1}.csv'
    part.to_csv(save_path, index=False)
    print(f"Cleaned data part {i+1} saved to: {save_path}")

    del part
    gc.collect()